In [2]:
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List

import pandas as pd
from ultralytics import YOLO


# =========================
# 你只需要改这里
# =========================
@dataclass
class EvalConfig:
    model_paths: List[str]
    data_yamls: List[str]
    split: str = "val"

    # 你要的 confidence 列表
    conf_list: List[float] = None

    # 评估参数（建议论文里固定）
    imgsz: int = 640
    batch: int = 16       # 默认16
    device: str = "0"     # "cpu" or "0"
    half: bool = True     # 默认True
    iou: float = 0.5      # 默认0.7
    max_det: int = 300    # 默认300

    # 输出目录
    out_dir: str = "./eval_tables"

    # 是否输出 per-class mAP (mAP50-95 per class)
    include_per_class: bool = True


CFG = EvalConfig(
    model_paths=[
        # "/workspace/_ty/02_models/best_models/04_swd_hbb/model_v2_4datasets_noAug_seed0_yolo11s_data_split_custom_8.pt",
        "/workspace/_ty/03_code/05_train_model/yolo/output/swd_model_v4_7datasets_null_image_full_noAug_seed42/yolo11n.pt_custom7null_cv1_ms2_0809-0823_10_ok_4/weights/best.pt",
    ],
    data_yamls=[
        "/workspace/_ty/03_code/05_train_model/yolo/yaml/data_split_custom.yaml",
    ],
    split="val",
    conf_list=[0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.91, 0.92, 0.93],
    imgsz=640,
    batch=16,
    device="0",
    half=True,
    iou=0.5,
    max_det=300,
    out_dir="./eval_tables",
    include_per_class=True,
)


# =========================
# 小工具
# =========================
def safe_get(obj: Any, attr: str, default=None):
    return getattr(obj, attr, default)


def to_float(x, default=None):
    try:
        return float(x)
    except Exception:
        return default


def f1(p: float | None, r: float | None) -> float | None:
    if p is None or r is None:
        return None
    if p + r == 0:
        return 0.0
    return 2 * p * r / (p + r)


def extract_row(metrics, model_path: str, data_yaml: str, split: str, conf: float) -> Dict[str, Any]:
    box = safe_get(metrics, "box", None)

    mp = to_float(safe_get(box, "mp", None))   # mean precision
    mr = to_float(safe_get(box, "mr", None))   # mean recall

    row: Dict[str, Any] = {
        "Model": Path(model_path).name,
        "DataYAML": Path(data_yaml).name,
        "Split": split,
        "Conf": conf,
        "Precision": mp,
        "Recall": mr,
        "F1": f1(mp, mr),
        "mAP50-95": to_float(safe_get(box, "map", None)),
        "mAP50": to_float(safe_get(box, "map50", None)),
        "mAP75": to_float(safe_get(box, "map75", None)),
    }

    if CFG.include_per_class:
        maps = safe_get(box, "maps", None)          # per-class mAP50-95
        names = safe_get(metrics, "names", None)    # id -> name
        if maps is not None and names is not None:
            if isinstance(names, dict):
                for cid, cname in names.items():
                    cid_int = int(cid)
                    if cid_int < len(maps):
                        row[f"mAP50-95[{cname}]"] = to_float(maps[cid_int])
            else:
                for cid, cname in enumerate(names):
                    if cid < len(maps):
                        row[f"mAP50-95[{cname}]"] = to_float(maps[cid])

    return row


def main():
    out_dir = Path(CFG.out_dir) / CFG.split
    out_dir.mkdir(parents=True, exist_ok=True)

    rows: List[Dict[str, Any]] = []

    for data_yaml in CFG.data_yamls:
        for model_path in CFG.model_paths:
            model = YOLO(model_path)

            for conf in CFG.conf_list:
                print(f"\n=== Evaluating ===")
                print(f"Model: {model_path}")
                print(f"Data : {data_yaml} | split={CFG.split} | conf={conf}")

                metrics = model.val(
                    data=data_yaml,
                    split=CFG.split,
                    imgsz=CFG.imgsz,
                    batch=CFG.batch,
                    device=CFG.device,
                    half=CFG.half,
                    conf=conf,          # <<< 关键：逐个conf跑
                    iou=CFG.iou,
                    max_det=CFG.max_det,
                    verbose=False,
                    plots=False,
                    workers=0,
                )

                rows.append(extract_row(metrics, model_path, data_yaml, CFG.split, conf))

    df = pd.DataFrame(rows)

    # 排序：更像论文表
    df = df.sort_values(["Model", "DataYAML", "Split", "Conf"]).reset_index(drop=True)

    # 数值列统一保留3位小数（论文常用）
    for c in df.columns:
        if c in ("Model", "DataYAML", "Split"):
            continue
        if pd.api.types.is_numeric_dtype(df[c]):
            df[c] = df[c].round(3)

    # 导出（long-format：每行一个conf，很适合论文）
    csv_path = out_dir / "metrics_by_conf.csv"
    xlsx_path = out_dir / "metrics_by_conf.xlsx"
    tex_path = out_dir / "metrics_by_conf.tex"

    df.to_csv(csv_path, index=False)
    df.to_excel(xlsx_path, index=False)

    latex_df = df.copy()
    latex = latex_df.to_latex(index=False, escape=True)
    tex_path.write_text(latex, encoding="utf-8")

    # 额外：生成“宽表”（一眼对比每个conf的P/R/F1）
    # 对论文里一个模型一张表很友好
    wide_cols = ["Precision", "Recall", "F1", "mAP50-95", "mAP50", "mAP75"]
    wide = df.pivot_table(
        index=["Model", "DataYAML", "Split"],
        columns="Conf",
        values=wide_cols,
        aggfunc="first",
    )
    wide.columns = [f"{metric}@{conf}" for metric, conf in wide.columns]
    wide = wide.reset_index()

    wide_csv = out_dir / "metrics_by_conf_wide.csv"
    wide_xlsx = out_dir / "metrics_by_conf_wide.xlsx"
    wide_tex = out_dir / "metrics_by_conf_wide.tex"

    wide.to_csv(wide_csv, index=False)
    wide.to_excel(wide_xlsx, index=False)
    wide_tex.write_text(wide.to_latex(index=False, escape=True), encoding="utf-8")

    print("\n✅ Done.")
    print(f"- Long CSV : {csv_path.resolve()}")
    print(f"- Long XLSX: {xlsx_path.resolve()}")
    print(f"- Long TEX : {tex_path.resolve()}")
    print(f"- Wide CSV : {wide_csv.resolve()}")
    print(f"- Wide XLSX: {wide_xlsx.resolve()}")
    print(f"- Wide TEX : {wide_tex.resolve()}")


if __name__ == "__main__":
    main()



=== Evaluating ===
Model: /workspace/_ty/03_code/05_train_model/yolo/output/swd_model_v4_7datasets_null_image_full_noAug_seed42/yolo11n.pt_custom7null_cv1_ms2_0809-0823_10_ok_4/weights/best.pt
Data : /workspace/_ty/03_code/05_train_model/yolo/yaml/data_split_custom.yaml | split=val | conf=0.5
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15818MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 156.4±102.2 MB/s, size: 36.1 KB)


val: Scanning /workspace/_ty/03_code/05_train_model/yolo/data_custom/custom/val/labels.cache... 759 images, 0 backgrounds, 0 corrupt: 100%|██████████| 759/759 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 48/48 [00:04<00:00, 11.26it/s]

                   all        759        785      0.991      0.669       0.83      0.732
Speed: 0.1ms preprocess, 3.0ms inference, 0.0ms loss, 0.4ms postprocess per image

=== Evaluating ===
Model: /workspace/_ty/03_code/05_train_model/yolo/output/swd_model_v4_7datasets_null_image_full_noAug_seed42/yolo11n.pt_custom7null_cv1_ms2_0809-0823_10_ok_4/weights/best.pt
Data : /workspace/_ty/03_code/05_train_model/yolo/yaml/data_split_custom.yaml | split=val | conf=0.6
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15818MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2682.4±743.3 MB/s, size: 29.3 KB)



val: Scanning /workspace/_ty/03_code/05_train_model/yolo/data_custom/custom/val/labels.cache... 759 images, 0 backgrounds, 0 corrupt: 100%|██████████| 759/759 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 48/48 [00:01<00:00, 27.26it/s]

                   all        759        785      0.988      0.535      0.763      0.675
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.2ms postprocess per image

=== Evaluating ===
Model: /workspace/_ty/03_code/05_train_model/yolo/output/swd_model_v4_7datasets_null_image_full_noAug_seed42/yolo11n.pt_custom7null_cv1_ms2_0809-0823_10_ok_4/weights/best.pt
Data : /workspace/_ty/03_code/05_train_model/yolo/yaml/data_split_custom.yaml | split=val | conf=0.7
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15818MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3585.3±1413.3 MB/s, size: 40.0 KB)


...


val: Scanning /workspace/_ty/03_code/05_train_model/yolo/data_custom/custom/val/labels.cache... 759 images, 0 backgrounds, 0 corrupt: 100%|██████████| 759/759 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 48/48 [00:01<00:00, 30.15it/s]

                   all        759        785          0          0          0          0
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.1ms postprocess per image

=== Evaluating ===
Model: /workspace/_ty/03_code/05_train_model/yolo/output/swd_model_v4_7datasets_null_image_full_noAug_seed42/yolo11n.pt_custom7null_cv1_ms2_0809-0823_10_ok_4/weights/best.pt
Data : /workspace/_ty/03_code/05_train_model/yolo/yaml/data_split_custom.yaml | split=val | conf=0.93
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15818MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3449.2±1928.4 MB/s, size: 46.3 KB)



val: Scanning /workspace/_ty/03_code/05_train_model/yolo/data_custom/custom/val/labels.cache... 759 images, 0 backgrounds, 0 corrupt: 100%|██████████| 759/759 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 48/48 [00:01<00:00, 30.34it/s]

                   all        759        785          0          0          0          0
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.1ms postprocess per image



✅ Done.
- Long CSV : /workspace/_ty/03_code/05_train_model/yolo/eval_tables/val/metrics_by_conf.csv
- Long XLSX: /workspace/_ty/03_code/05_train_model/yolo/eval_tables/val/metrics_by_conf.xlsx
- Long TEX : /workspace/_ty/03_code/05_train_model/yolo/eval_tables/val/metrics_by_conf.tex
- Wide CSV : /workspace/_ty/03_code/05_train_model/yolo/eval_tables/val/metrics_by_conf_wide.csv
- Wide XLSX: /workspace/_ty/03_code/05_train_model/yolo/eval_tables/val/metrics_by_conf_wide.xlsx
- Wide TEX : /workspace/_ty/03_code/05_train_model/yolo/eval_tables/val/metrics_by_conf_wide.tex


### v2

In [9]:
from pathlib import Path

root = Path("/workspace/_ty/03_code/05_train_model/yolo/output/swd_model_v4_7datasets_null_image_20pct_noAug_seed42")

results = []
results_name = []

for best_pt in root.rglob("weights/best.pt"):
    name = best_pt.parents[1].name  # weights 的上一级
    results.append(str(best_pt))
    results_name.append(name)

# 示例打印
for r in results:
    print(f"\"{r}\",")
for n in results_name:
    print(n)

"/workspace/_ty/03_code/05_train_model/yolo/output/swd_model_v4_7datasets_null_image_20pct_noAug_seed42/yolo11n.pt_custom7null_cv5_ms2_0726-0809_13_ok_4/weights/best.pt",
"/workspace/_ty/03_code/05_train_model/yolo/output/swd_model_v4_7datasets_null_image_20pct_noAug_seed42/yolo11s.pt_custom7null_cv5_ms2_0726-0809_13_ok_4/weights/best.pt",
"/workspace/_ty/03_code/05_train_model/yolo/output/swd_model_v4_7datasets_null_image_20pct_noAug_seed42/yolo11s.pt_custom7null_cv5_ms2_0726-0809_13_ok_16/weights/best.pt",
"/workspace/_ty/03_code/05_train_model/yolo/output/swd_model_v4_7datasets_null_image_20pct_noAug_seed42/yolo11s.pt_custom7null_cv1_ms2_0809-0823_10_ok_16/weights/best.pt",
"/workspace/_ty/03_code/05_train_model/yolo/output/swd_model_v4_7datasets_null_image_20pct_noAug_seed42/yolo11n.pt_custom7null_cv4_ms1_0605-0621_40_ok_4/weights/best.pt",
"/workspace/_ty/03_code/05_train_model/yolo/output/swd_model_v4_7datasets_null_image_20pct_noAug_seed42/yolo11s.pt_custom7null_cv2_ms1_0710-072

In [12]:
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List, Optional

import pandas as pd
from ultralytics import YOLO


# =========================
# 配置
# =========================
@dataclass
class EvalConfig:
    model_paths: List[str]
    data_yamls: List[str]
    split: str = "val"

    conf_list: List[float] = None
    iou_threshold_list: List[float] = None

    imgsz: int = 640
    batch: int = 16
    device: str = "0"
    half: bool = True
    max_det: int = 300

    out_dir: str = "./eval_tables"


CFG = EvalConfig(
    model_paths=[
        "/workspace/_ty/03_code/05_train_model/yolo/output/swd_model_v4_7datasets_null_image_20pct_noAug_seed42/yolo11n.pt_custom7null_cv1_ms2_0809-0823_10_ok_8/weights/best.pt",
    ],
    data_yamls=[
        "/workspace/_ty/03_code/05_train_model/yolo/yaml/data_split_custom.yaml",
    ],
    split="test",
    conf_list=[0],
    iou_threshold_list=[0.5],  # 你想扫哪些 IoU 阈值
    imgsz=640,
    batch=16,
    device="0",
    half=True,
    max_det=300,
    out_dir="./eval_tables",
)

CFG = EvalConfig(
    model_paths=results,
    data_yamls=[
        "/workspace/_ty/03_code/05_train_model/yolo/yaml/data_split_custom.yaml",
    ],
    split="test",
    conf_list=[0, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9],
    iou_threshold_list=[0.5],  # 你想扫哪些 IoU 阈值
    imgsz=640,
    batch=16,
    device="0",
    half=True,
    max_det=300,
    out_dir="./eval_tables",
)


# =========================
# 小工具
# =========================
def safe_get(obj: Any, attr: str, default=None):
    return getattr(obj, attr, default)


def to_float(x, default=None):
    try:
        return float(x)
    except Exception:
        return default


def f1(p: Optional[float], r: Optional[float]) -> Optional[float]:
    if p is None or r is None:
        return None
    if p + r == 0:
        return 0.0
    return 2 * p * r / (p + r)


def extract_tp_fp_fn(metrics) -> Dict[str, Optional[int]]:
    """
    从 Ultralytics val() 的 confusion_matrix 里提取 TP/FP/FN。
    常见定义（YOLO confusion matrix）：
      - 前 nc 行/列：各类别
      - 最后一行：GT 为 background（没有 GT），但预测成某类 -> FP
      - 最后一列：预测为 background，但 GT 有对象 -> FN
    """
    cm_obj = safe_get(metrics, "confusion_matrix", None)
    if cm_obj is None:
        return {"tp": None, "fp": None, "fn": None}

    m = safe_get(cm_obj, "matrix", None)
    if m is None:
        return {"tp": None, "fp": None, "fn": None}

    # m 是 numpy array
    try:
        nc = m.shape[0] - 1
        tp = int(m[:nc, :nc].trace())
        fp = int(m[nc, :nc].sum())
        fn = int(m[:nc, nc].sum())
        return {"tp": tp, "fp": fp, "fn": fn}
    except Exception:
        return {"tp": None, "fp": None, "fn": None}


def det_accuracy(tp: Optional[int], fp: Optional[int], fn: Optional[int]) -> Optional[float]:
    """
    检测任务常用的“accuracy”（无 TN）：
      acc = TP / (TP + FP + FN)
    """
    if tp is None or fp is None or fn is None:
        return None
    denom = tp + fp + fn
    if denom == 0:
        return None
    return tp / denom


def extract_row(metrics, model_path: str, data_yaml: str, split: str, conf: float, iou_thr: float) -> Dict[str, Any]:
    box = safe_get(metrics, "box", None)

    precision = to_float(safe_get(box, "mp", None))  # mean precision
    recall = to_float(safe_get(box, "mr", None))      # mean recall
    f1v = f1(precision, recall)

    map_5095 = to_float(safe_get(box, "map", None))   # mAP50-95
    map_50 = to_float(safe_get(box, "map50", None))   # mAP50（可选）

    counts = extract_tp_fp_fn(metrics)
    tp, fp, fn = counts["tp"], counts["fp"], counts["fn"]

    row: Dict[str, Any] = {
        "Model": Path(model_path),
        "DataYAML": Path(data_yaml).name,
        "Split": split,

        "conf": conf,  # ✅ 加上这一行

        "accuracy": det_accuracy(tp, fp, fn),
        "iou_threshold": iou_thr,
        "precision": precision,
        "recall": recall,
        "f1_score": f1v,

        # 你要 map / mar：这里 map= mAP50-95，mar= mr（mean recall）
        "map": map_5095,
        "mar": recall,

        "true_positives": tp,
        "false_positives": fp,
        "false_negatives": fn,

        # 如果你也想保留 mAP50，解注释即可：
        # "map50": map_50,
    }
    return row


def main():
    out_dir = Path(CFG.out_dir) / CFG.split
    out_dir.mkdir(parents=True, exist_ok=True)

    rows: List[Dict[str, Any]] = []

    for data_yaml in CFG.data_yamls:
        for model_path in CFG.model_paths:
            model = YOLO(model_path)

            for iou_thr in CFG.iou_threshold_list:
                for conf in CFG.conf_list:
                    print(f"\n=== Evaluating ===")
                    print(f"Model: {model_path}")
                    print(f"Data : {data_yaml} | split={CFG.split} | conf={conf} | iou={iou_thr}")

                    metrics = model.val(
                        data=data_yaml,
                        split=CFG.split,
                        imgsz=CFG.imgsz,
                        batch=CFG.batch,
                        device=CFG.device,
                        half=CFG.half,
                        conf=conf,          # 这里是“过滤阈值”，会影响 P/R/F1 以及 confusion matrix
                        iou=iou_thr,
                        max_det=CFG.max_det,
                        verbose=False,
                        plots=False,
                        workers=0,
                    )

                    rows.append(extract_row(metrics, model_path, data_yaml, CFG.split, conf, iou_thr))

    df = pd.DataFrame(rows)

    # 排序更清晰
    df = df.sort_values(["Model", "DataYAML", "Split", "iou_threshold", "precision"], ascending=[True, True, True, True, False]).reset_index(drop=True)

    # 数值列保留 6 位（你也可以改成 3 位）
    for c in df.columns:
        if c in ("Model", "DataYAML", "Split"):
            continue
        if pd.api.types.is_numeric_dtype(df[c]):
            df[c] = df[c].round(6)

    csv_path = out_dir / "metrics_with_counts.csv"
    xlsx_path = out_dir / "metrics_with_counts.xlsx"
    df.to_csv(csv_path, index=False)
    df.to_excel(xlsx_path, index=False)

    print("\n✅ Done.")
    print(f"- CSV : {csv_path.resolve()}")
    print(f"- XLSX: {xlsx_path.resolve()}")


if __name__ == "__main__":
    main()


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 119/119 [00:04<00:00, 25.74it/s]

                   all       1900       1610      0.953      0.793      0.883      0.853
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.2ms postprocess per image

=== Evaluating ===
Model: /workspace/_ty/03_code/05_train_model/yolo/output/swd_model_v4_7datasets_null_image_20pct_noAug_seed42/yolo11n.pt_custom7null_cv4_ms1_0605-0621_40_ok_16/weights/best.pt
Data : /workspace/_ty/03_code/05_train_model/yolo/yaml/data_split_custom.yaml | split=test | conf=0.6 | iou=0.5
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15818MiB)


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3208.4±708.1 MB/s, size: 42.8 KB)


val: Scanning /workspace/_ty/03_code/05_train_model/yolo/data_v2_20pct/ms1_0726-0809_11_ok/labels.cache... 1900 images, 317 backgrounds, 0 corrupt: 100%|██████████| 1900/1900 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 119/119 [00:04<00:00, 26.08it/s]

                   all       1900       1610      0.958      0.766      0.871      0.842
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.2ms postprocess per image

=== Evaluating ===
Model: /workspace/_ty/03_code/05_train_model/yolo/output/swd_model_v4_7datasets_null_image_20pct_noAug_seed42/yolo11n.pt_custom7null_cv4_ms1_0605-0621_40_ok_16/weights/best.pt
Data : /workspace/_ty/03_code/05_train_model/yolo/yaml/data_split_custom.yaml | split=test | conf=0.7 | iou=0.5
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15818MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3142.1±425.2 MB/s, size: 45.5 KB)


...

                   all       1900       1610      0.985      0.682      0.837      0.813
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.2ms postprocess per image

=== Evaluating ===
Model: /workspace/_ty/03_code/05_train_model/yolo/output/swd_model_v4_7datasets_null_image_20pct_noAug_seed42/yolo11s.pt_custom7null_cv1_ms2_0809-0823_10_ok_8/weights/best.pt
Data : /workspace/_ty/03_code/05_train_model/yolo/yaml/data_split_custom.yaml | split=test | conf=0.85 | iou=0.5
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15818MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2715.9±630.8 MB/s, size: 40.1 KB)



val: Scanning /workspace/_ty/03_code/05_train_model/yolo/data_v2_20pct/ms1_0726-0809_11_ok/labels.cache... 1900 images, 317 backgrounds, 0 corrupt: 100%|██████████| 1900/1900 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 119/119 [00:05<00:00, 22.10it/s]

                   all       1900       1610      0.991      0.592      0.793      0.772
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.2ms postprocess per image

=== Evaluating ===
Model: /workspace/_ty/03_code/05_train_model/yolo/output/swd_model_v4_7datasets_null_image_20pct_noAug_seed42/yolo11s.pt_custom7null_cv1_ms2_0809-0823_10_ok_8/weights/best.pt
Data : /workspace/_ty/03_code/05_train_model/yolo/yaml/data_split_custom.yaml | split=test | conf=0.9 | iou=0.5
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15818MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3059.4±388.1 MB/s, size: 38.4 KB)



val: Scanning /workspace/_ty/03_code/05_train_model/yolo/data_v2_20pct/ms1_0726-0809_11_ok/labels.cache... 1900 images, 317 backgrounds, 0 corrupt: 100%|██████████| 1900/1900 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 119/119 [00:05<00:00, 22.29it/s]

                   all       1900       1610          1       0.28       0.64      0.629
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.1ms postprocess per image

✅ Done.
- CSV : /workspace/_ty/03_code/05_train_model/yolo/eval_tables/test/metrics_with_counts.csv
- XLSX: /workspace/_ty/03_code/05_train_model/yolo/eval_tables/test/metrics_with_counts.xlsx
